# Geoapify Places 5 Cities

In [23]:
# Dependencies
from census import Census
import hvplot.pandas
import time
import requests
import json
import pandas as pd
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import the API key
from config_key import geoapify_key
from config_key import census_key

In [24]:
#c.acs5.tables()

In [25]:
# Create an instance of the Census library
# 
c = Census(
    census_key, 
    year=2021
)

# Run Census Search to retrieve data on all zip codes (2021 ACS5 Census)
census_data = c.acs5.get(
    (
        "B01003_001E",
        "B17001_002E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column renaming
census_df = census_df.rename(
    columns = {
        "B01003_001E": "Population",
        "B17001_002E": "Poverty Count",
        "zip code tabulation area": "Zipcode"
    }
)

In [26]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [27]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA","Monmouth, OR"]
#county = ["Bronx, NY","Napa, CA", "San Bernardino, CA","San Francisco, CA"]
#district = ["Bronx, NY","Napa, CA", "San Bernardino, CA","San Francisco, CA"]


#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]

# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#US_locations_df = pd.DataFrame({
                            #"cities": cities, 
                            #"latitude": latitude, 
                            #"longitude": longitude #,
#                            "district": district,
#                            "county": county
#                            }
#                            )

# Add empty columns to the DataFrame to store, supermarket (property name), distance, etc. 
#US_locations_df["supermarket"] = ""
#US_locations_df["distance"] = ""
#US_locations_df["postcode"] = ""
#US_locations_df["district"] = ""
#US_locations_df["latitude"] = ""
#US_locations_df["longitude"] = ""


# Set the parameters to search for supermarkets 
#categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16200

# Set up a params dictionary
#params = {
#    "categories":categories,
#    "apiKey":geoapify_key,
#    "limit": 100
#}

# Print a message to follow up the supermarket search
#print("Starting supermarket search...") 

# Iterate through the US_locations_df DataFrame
#for index, row in US_locations_df.iterrows():
#    # get latitude, longitude from the DataFrame
#    #longitude = row["longitude"]
#    #latitude = row["latitude"]
#    county = row["County"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#    params["filter"] = f"circle:{longitude},{latitude},{radius}"
#    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
#    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
#    response = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
#    response = response.json()

    # Grab the first supermarket from the results and store the name in the US_locations_df DataFrame
#    try:
#        US_locations_df.loc[index, "supermarket"] = response["features"][0]["properties"]["address_line1"]
#       US_locations_df.loc[index, "distance"] = response["features"][0]["properties"]["distance"]
#        US_locations_df.loc[index, "postcode"] = response["features"][0]["properties"]["postcode"]
        
#    except (KeyError, IndexError):
#        # If no supermarket is found, set the supermarket name as "No supermarket found".
#        US_locations_df.loc[index, "supermarket"] = "No supermarket found"
#        US_locations_df.loc[index, "distance"] = "No distance found"
#        US_locations_df.loc[index, "postcode"] = "No zipcode found"
      
    # Log the search results
#    print(f"{US_locations_df.loc[index, 'cities']} - nearest supermarket: {US_locations_df.loc[index, 'supermarket']} - zipcode: {US_locations_df.loc[index, 'postcode']}")

# Display sample data
#US_locations_df



In [28]:
#US_locations_df = US_locations_df.rename(columns={"postcode":"Zipcode"})
#US_locations_df

In [29]:
#batch API call
#url = "https://api.geoapify.com/v1/batch?apiKey=geoapify_key
#headers = {"Content-Type": "application/json"}
#data = '{"api":"/v2/places","params":{},"inputs":[{"id":"optional-input-id1","params":{}}]}'
      
#try:
#    resp = requests.post(url, headers=headers, data=data)
#    print(resp.json())
#except requests.exceptions.HTTPError as e:
#    print (e.response.text)

In [76]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=YOUR_API_KEY

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
          
#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]


# Set the parameters to search for supermarkets 
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
i = 0

# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[3]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionaty
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208


In [77]:
# convert response to json
supermarket_data = response.json()

# Print the json (pretty printed)
print(json.dumps(supermarket_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -122.4151303026334,
                    37.76908831549504
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Rainbow Grocery Coop",
                "address_line2": "1745 Folsom Street, San Francisco, CA 94103, United States of America",
                "categories": [
                    "building",
                    "building.commercial",
                    "commercial",
                    "commercial.supermarket",
                    "wheelchair",
                    "wheelchair.yes"
                ],
                "city": "San Francisco",
                "country": "United States",
                "country_code": "us",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                

In [78]:
# Extracting 'features' and 'properties' from JSON
features = supermarket_data.get("features", [])
rows = [] # Iterate through the results 

In [79]:

# Extracting results from JSON 
# results = supermarket_data.get("results", []) # Create a list to store rows 
supermarket_count = 1

for feature in features: 
    properties = feature.get("properties",{})
    # Extract relevant information
    name = properties.get("name") 
    lat = properties.get("lat") 
    lon = properties.get("lon") # Append data to the list
    postcode = properties.get("postcode")
    radius = properties.get("radius")
    city = properties.get("city")
    rows.append({"Name": name, "City": city, "Zipcode": postcode, "Latitude": lat, "Longitude": lon, "Supermarket Count": supermarket_count}) 
    # rows.append({"Name": name, "Latitude": lat, "Longitude": lon, "Supermarket Count": supermarket_count}) 
    # rows.append({"Name": name, "City": city, "Zipcode": postcode, "Latitude": lat, "Longitude": lon, "Supermarket Count": supermarket_count}) 
    #rows.append({"City": city, "Zipcode": postcode, "Supermarket Count": supermarket_count}) 

    
print(rows)

[{'Name': 'Rainbow Grocery Coop', 'City': 'San Francisco', 'Zipcode': '94103', 'Latitude': 37.769136700000004, 'Longitude': -122.41507506128232, 'Supermarket Count': 1}, {'Name': 'Nijiya Market', 'City': 'San Francisco', 'Zipcode': '94115', 'Latitude': 37.7852546, 'Longitude': -122.4310324, 'Supermarket Count': 1}, {'Name': 'Safeway', 'City': 'San Francisco', 'Zipcode': '94116', 'Latitude': 37.7433862, 'Longitude': -122.47411849296468, 'Supermarket Count': 1}, {'Name': 'Foods Co', 'City': 'San Francisco', 'Zipcode': '94124', 'Latitude': 37.7298078, 'Longitude': -122.39882094695228, 'Supermarket Count': 1}, {'Name': 'Safeway', 'City': 'San Francisco', 'Zipcode': '94131', 'Latitude': 37.74357125, 'Longitude': -122.43894408275185, 'Supermarket Count': 1}, {'Name': 'Canyon Market', 'City': 'San Francisco', 'Zipcode': '94131', 'Latitude': 37.7340644, 'Longitude': -122.43382389860392, 'Supermarket Count': 1}, {'Name': 'Whole Foods Market', 'City': 'San Francisco', 'Zipcode': '94107', 'Latitu

In [80]:
# Create a DataFrame 
supermarket_df = pd.DataFrame(rows) # Display the DataFrame 
print(supermarket_df) 
# should not return city other than Napa

                      Name           City Zipcode   Latitude   Longitude  \
0     Rainbow Grocery Coop  San Francisco   94103  37.769137 -122.415075   
1            Nijiya Market  San Francisco   94115  37.785255 -122.431032   
2                  Safeway  San Francisco   94116  37.743386 -122.474118   
3                 Foods Co  San Francisco   94124  37.729808 -122.398821   
4                  Safeway  San Francisco   94131  37.743571 -122.438944   
5            Canyon Market  San Francisco   94131  37.734064 -122.433824   
6       Whole Foods Market  San Francisco   94107  37.781150 -122.399779   
7           Mollie Stone's  San Francisco   94114  37.760666 -122.436559   
8           Grocery Outlet  San Francisco   94110  37.753080 -122.415953   
9               The Market  San Francisco   94103  37.776542 -122.416807   
10       Real Food Company  San Francisco   94109  37.796547 -122.421707   
11           Pacific Super  San Francisco   94112  37.711972 -122.452816   
12      Whol

In [81]:
supermarket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               20 non-null     object 
 1   City               20 non-null     object 
 2   Zipcode            20 non-null     object 
 3   Latitude           20 non-null     float64
 4   Longitude          20 non-null     float64
 5   Supermarket Count  20 non-null     int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ KB


In [82]:
supermarket_df.columns

Index(['Name', 'City', 'Zipcode', 'Latitude', 'Longitude',
       'Supermarket Count'],
      dtype='object')

In [83]:
#US_locations_df.columns

In [84]:
#supermkt_census_df = pd.merge(supermarket_df, US_locations_df, how="left", on=["Zipcode","Zipcode"])

# Display sample data
#supermkt_census_df

supermarket_totals_df = supermarket_df.groupby(['City', 'Zipcode']).size().reset_index(name='Supermarket Count')
# supermarket_totals_df = pd.DataFrame[{"City":supermarket_df["City"],"Zipcode":supermarket_df["Zipcode"],"Supermarket Count":supermarket_df["Supermarket Count"]}].groupby["Zipcode"]
supermarket_totals_df

,City,Zipcode,Supermarket Count
0,San Francisco,94103,2
1,San Francisco,94107,2
2,San Francisco,94109,1
3,San Francisco,94110,2
4,San Francisco,94111,1
5,San Francisco,94112,1
6,San Francisco,94114,2
7,San Francisco,94115,1
8,San Francisco,94116,1
9,San Francisco,94118,1


In [85]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [86]:
# Merge datasets on Zipcodes
supermarkets_density_final = pd.merge(
    supermarket_totals_df,
    census_df,
    how = "left",
    on = ["Zipcode", "Zipcode"]
)

In [87]:
supermarkets_density_final

,City,Zipcode,Supermarket Count,Population,Poverty Count
0,San Francisco,94103,2,32430.0,5202.0
1,San Francisco,94107,2,28607.0,2206.0
2,San Francisco,94109,1,56114.0,6850.0
3,San Francisco,94110,2,70859.0,6939.0
4,San Francisco,94111,1,4707.0,858.0
5,San Francisco,94112,1,84477.0,7560.0
6,San Francisco,94114,2,34474.0,1979.0
7,San Francisco,94115,1,34193.0,4195.0
8,San Francisco,94116,1,45510.0,2680.0
9,San Francisco,94118,1,41456.0,2538.0


In [88]:
# Add a Supermarket Count column (Supermarkets/Population)
Supermarket_Total = supermarkets_density_final["Supermarket Count"].sum().astype(int)
Population_Total = supermarkets_density_final["Population"].astype(int).groupby(["Zipcode"])
Supermarket_Density = Supermarket_Total / Population_Total

# Configure the final DataFrame
supermarkets_density_final = supermarkets_density_final[
    [
        "postcode",
        "population",
        "supermarket"
    ]
]

# Display DataFrame length and sample data
# print(f"Number of rows in the DataFrame: {len(supermarkets_density_final)}")
supermarkets_density_final.head()

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer